# Rewriting TransferFuncList

In [1]:
%load_ext autoreload
import sys
sys.path.append("../../..")

In [2]:
%autoreload

import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10,10]

# SET THIS TO TRUE TO USE LATEX
matplotlib.rcParams['text.usetex'] = False

from astropy.io import fits

import numpy as np
import copy
import pickle

import darkhistory.physics as phys
import darkhistory.utilities as utils
import darkhistory.spec.spectools as spectools
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.transferfunclist as tflist
from darkhistory.spec.spectrum import Spectrum
from darkhistory.spec.spectra import Spectra
import darkhistory.history.tla as tla

from scipy.interpolate import interp1d
from scipy.interpolate import interp2d

from tqdm import tqdm_notebook as tqdm

from darkhistory.electrons.ics.ics_spectrum import ics_spec
from darkhistory.electrons.ics.ics_engloss_spectrum import engloss_spec
from darkhistory.electrons.ics.ics_cooling import get_ics_cooling_tf

from darkhistory.low_energy.lowE_deposition import compute_fs
import main

from tqdm import tqdm_notebook as tqdm

# np.set_printoptions(threshold=np.nan)

In [6]:
standard_tfs = False
direc = '/Users/hongwan/Dropbox (MIT)/Photon Deposition/tfs_20190422/'

flt32 = True

if not flt32:
    flt32_str = ''
else:
    flt32_str = '_flt32'

string_fixed_xe = 'standard_xe_xHe_full_rs'+flt32_str
string_with_xHe_0 = 'xHe0'+flt32_str
string_with_xHe = 'full_xHe'+flt32_str

################# Standard xe Transfer Functions ##############################
# This is a TransferFuncList
highengphot_tflist_fixed_xe = pickle.load(open(direc+'tfunclist_photspec_60eV_complete_'+string_fixed_xe+'.raw','rb'))
highengphot_tflist_fixed_xe = highengphot_tflist_fixed_xe[0]

################# xHe=0 Transfer Functions ####################################
highengphot_tflist_xHe_0 = pickle.load(open(direc+'tfunclist_photspec_60eV_complete_'+string_with_xHe_0+'.raw', 'rb'))

xes_xHe_0 = 0.5 + 0.5*np.tanh(
    [-5., -4.1, -3.2, -2.3, -1.8, -1.4, -1, -.8, -0.65, -0.5, -.24, 0, .2, 0.4, .7, 1.3, 2.2, 3.1, 4]
)



################# Full xHe-Dependent Transfer Functions #######################

highengphot_tflist_arr_with_xHe = pickle.load(open(direc+'tfunclist_photspec_60eV_complete_'+string_with_xHe+'.raw', 'rb'))

xes_with_xHe = 0.5 + 0.5*np.tanh(
    np.array(
        [-5.0, -2.3, -0.5, 0.0, 0.4, 2.2, 4.0]
    )
)

x_arr_with_xHe = np.array([[[xH, xHe] for xHe in xes_with_xHe] for xH in xes_with_xHe])


In [9]:
# highengphot_tflist_arr_with_xHe
# highengphot_tflist_xHe_0
highengphot_tflist_fixed_xe

In [10]:

highengphot_xHe_0_tflistarr = tflist.TransferFuncListArray(highengphot_tflist_xHe_0, xes_xHe_0) 
highengphot_full_xHe_tflistarr = tflist.TransferFuncListArray(highengphot_tflist_arr_with_xHe, x_arr_with_xHe)


../../../darkhistory/spec/transferfunclist.py:473: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  ) for tflist_xHe_arr in tflist_arr


In [12]:


list_of_tflistarr = [
        highengphot_full_xHe_tflistarr, 
        highengphot_xHe_0_tflistarr, 
        highengphot_tflist_fixed_xe
    ]

a = tflist.TransferFuncInterp(list_of_tflistarr, rs_nodes=[30, 1600])

noninterp


In [13]:

a.get_tf(0.5, 0.5, 300)

In [9]:
direc = '/Users/hongwan/Dropbox (MIT)/Photon Deposition/tfs_20190422/'

flt32 = True

if not flt32:
    flt32_str = ''
else:
    flt32_str = '_flt32'

string_fixed_xe = 'standard_xe_xHe_full_rs'
string_with_xHe_0 = 'xHe0'
string_with_xHe = 'full_xHe'

xes_xHe_0 = 0.5 + 0.5*np.tanh(
    [-5., -4.1, -3.2, -2.3, -1.8, -1.4, -1, -.8, -0.65, -0.5, -.24, 0, .2, 0.4, .7, 1.3, 2.2, 3.1, 4]
)

xes_with_xHe = 0.5 + 0.5*np.tanh(
    np.array(
        [-5.0, -2.3, -0.5, 0.0, 0.4, 2.2, 4.0]
    )
)

x_arr_with_xHe = np.array([[[xH, xHe] for xHe in xes_with_xHe] for xH in xes_with_xHe])


# highengdep_arr = pickle.load(
#             open(
#                 direc+"highdeposited_60eV_complete_"
#                 +string_with_xHe+flt32_str+".raw", "rb"
#             )
#         )

In [10]:
%autoreload

import setup_stuff

direc_arr = ['/Users/hongwan/Dropbox (MIT)/Photon Deposition/tfs_20190422/' for i in np.arange(3)]
xes = [x_arr_with_xHe, xes_xHe_0, None]
string_arr = [string_with_xHe+flt32_str, string_with_xHe_0+flt32_str, string_fixed_xe+flt32_str]

# direc_arr = ['/Users/hongwan/Dropbox (MIT)/Photon Deposition/tfs_20190120/' for i in np.arange(2)]
# xes = [xes_xHe_0, None]
# string_arr = [string_with_xHe_0, string_fixed_xe]

CMB_subtracted = False

(highengphot_tf_interp,
     lowengphot_tf_interp,
     lowengelec_tf_interp,
     highengdep_interp,
     CMB_engloss_interp
    ) = setup_stuff.load_trans_funcs(
        direc_arr, xes, 
        rs_nodes = np.array([40, 1600]),
#         rs_nodes = np.array([40, 1600]),
        string_arr=string_arr, CMB_subtracted=CMB_subtracted)


Loading transfer functions, set 0
Loaded high energy photons...
Low energy photons...
Low energy electrons...
High energy deposition...
CMB losses.
Transfer function lists set  0  complete.
Padding transfer functions...
High energy photons...
Low energy photons...
Low energy electrons...
High energy deposition...
CMB losses...
Creating array objects...
Generating TransferFuncInterp object...
Loading transfer functions, set 1
Loaded high energy photons...
Low energy photons...
Low energy electrons...
High energy deposition...
CMB losses.
Transfer function lists set  1  complete.
Padding transfer functions...
High energy photons...
Low energy photons...
Low energy electrons...
High energy deposition...
CMB losses...
Creating array objects...
Generating TransferFuncInterp object...
Loading transfer functions, set 2
Loaded high energy photons...
Low energy photons...
Low energy electrons...
High energy deposition...
CMB losses.
Transfer function lists set  2  complete.
Padding transfer fun

In [18]:
a = highengphot_tf_interp.get_tf(0.5, 0.9, 200)

In [19]:
a.grid_vals.dtype

dtype('float64')

In [11]:
data_direc = '/Users/hongwan/Dropbox (MIT)/Photon Deposition/DarkHistory_data/'

if flt32:

    pickle.dump(highengphot_tf_interp, open(data_direc+'highengphot_tf_interp_flt32.raw','wb'))
    pickle.dump(lowengphot_tf_interp,  open(data_direc+'lowengphot_tf_interp_flt32.raw' ,'wb'))
    pickle.dump(lowengelec_tf_interp,  open(data_direc+'lowengelec_tf_interp_flt32.raw' ,'wb'))
    # pickle.dump(highengdep_interp, data_direc+'highengde_tf_interp_flt32.raw')
    # pickle.dump(CMB_engloss_interp, data_direc+'highengphot_tf_interp_flt32.raw')

OSError: [Errno 22] Invalid argument